### Step1. 라이브러리 가져오기

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
import visdom

vis = visdom.Visdom()
vis.close(env='main')

Setting up a new session...


''

### loss_tracker 정의!

CNN 학습시에 Loss function을 자동으로 찍으려는 의도!

In [3]:
def loss_tracker(loss_plot, loss_value, num):
    vis.line(X=num,
            Y=loss_value,
            win=loss_plot,
            update='append'
            )

### Step2. GPU 사용 설정

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

### Step3. 학습 파라미터 설정하기

In [5]:
learning_rate = 0.001
training_epochs = 7 # 저사양 노트북으로 인하여...
batch_size = 32

### Step4. 데이터셋 가져오고 데이터로더 만들기

In [6]:
mnist_train = datasets.MNIST(root='MNIST_data/',
                            train=True,
                            transform = transforms.ToTensor(),
                            download=True)

mnist_test = datasets.MNIST(root='MNIST_data/',
                           train=False,
                           transform = transforms.ToTensor(),
                           download=True)

In [7]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size = batch_size, 
                                         shuffle=True,
                                         drop_last=True)

### Step5. 모델 만들기

In [8]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [9]:
model = CNN().to(device)

value = (torch.Tensor(1, 1, 28, 28)).to(device)
print(model(value).shape) # 오류 검수차 

torch.Size([1, 10])


### Step6. Loss Function, Optimizer 선언하기

In [10]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

### Plotting!

In [11]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(), opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

### Step7. Loss_tracker 사용해서 훈련시키기

In [ ]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {}] cost = {}' .format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))

print('학습 끝!')

[Epoch: 1] cost = 0.12071861326694489
[Epoch: 2] cost = 0.04059869423508644
[Epoch: 3] cost = 0.029636917635798454


In [ ]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())